<a href="https://colab.research.google.com/github/gabrielycarrari/tcc/blob/pre-processing/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook para Pré-processamento

# Importações

In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
from huggingface_hub import login
from google.colab import userdata


In [ ]:
login(token=userdata.get('HUGGINGFACE_TOKEN'))

In [ ]:
# Carregar o dataset Common Voice 17
dataset = load_dataset("mozilla-foundation/common_voice_17_0", "pt", split='train', trust_remote_code=True)